This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 4601 W WOOLWORTH AV, Milwaukee, WI - [43.13310406251304, -87.97014909980484]
Parsed: 5385 N GREEN BAY AV, Milwaukee, WI - [43.11489740519164, -87.93399747326869]
Parsed: 4136 N GREEN BAY AV, Milwaukee, WI - [43.09241987705574, -87.92443329805548]
Parsed: 7140 N 41ST ST, Milwaukee, WI - [43.14666427116396, -87.96304284520212]
Parsed: 7260 N 43RD ST, Milwaukee, WI - [43.14985879154153, -87.96528176815933]
Parsed: 5385 N GREEN BAY AV, Milwaukee, WI - [43.11489740519164, -87.93399747326869]
Parsed: 4142 N 12TH ST, Milwaukee, WI - [43.092314077990665, -87.92570588631791]
Parsed: 4338 N 21ST ST, Milwaukee, WI - [43.09618699056494, -87.93813424712926]
Parsed: 6043-C N TEUTONIA AV, Milwaukee, WI - [43.12749506819215, -87.95208493439858]
Parsed: 4573 N 28TH ST, Milwaukee, WI - [43.10055933527613, -87.94798159925536]
Parsed: 2446 W ATKINSON AV, Milwaukee, WI - [43.09444081101032, -87.9429047129957]
Parsed: 6859 N 43RD ST, Milwaukee, WI - [43.142463115182636, -87.96578809925535]
Parsed: 4

Parsed: 3540 S 43RD ST #103, Milwaukee, WI - [42.98012536940969, -87.96796642570294]
Parsed: 3220 S 49TH ST, Milwaukee, WI - [42.98614158090484, -87.97533294460172]
Parsed: 3648 S 84TH ST, Milwaukee, WI - [42.97806841326678, -88.01769648456367]
Parsed: 4014 S 73RD ST, Milwaukee, WI - [42.971571303912526, -88.00469295571023]
Parsed: 3277 S 27TH ST, Milwaukee, WI - [42.98532968028907, -87.9484498677004]
Parsed: 3783 S 48TH ST, Milwaukee, WI - [42.976428496797354, -87.974771481447]
Parsed: 3355 S 27TH ST, Milwaukee, WI - [42.98489091248832, -87.94843751889785]
Parsed: 3165 S 36TH ST, Milwaukee, WI - [42.986925586733236, -87.95943905539828]
Parsed: 4005 W OKLAHOMA AV, Milwaukee, WI - [42.98835250562685, -87.96481077407813]
Parsed: 3223 S 37TH ST, Milwaukee, WI - [42.98596426311386, -87.96066093883066]
Parsed: 3553 S 41ST ST #301, Milwaukee, WI - [42.98000399945174, -87.9657459417735]
Parsed: 3701 S 27TH ST, Milwaukee, WI - [42.9772934246089, -87.94859075166657]
Parsed: 3124 S 51ST ST, Milw

Parsed: 354 E NATIONAL AV, Milwaukee, WI - [43.023242482110845, -87.90670192339432]
Parsed: 1656 S 8TH ST, Milwaukee, WI - [43.01291624562873, -87.92121393349322]
Parsed: 1425 W GREENFIELD AV, Milwaukee, WI - [43.01701547013696, -87.93014327699233]
Parsed: 838 S 2ND ST, Milwaukee, WI - [43.02233966472389, -87.912615407958]
Parsed: 1657 S 17TH ST, Milwaukee, WI - [43.01302746841581, -87.93424397359829]
Parsed: 2615 S 14TH ST, Milwaukee, WI - [42.99722392339434, -87.92995806261165]
Parsed: 117 W WALKER ST, Milwaukee, WI - [43.02211554674261, -87.91194166763806]
Parsed: 1115 W ORCHARD ST, Milwaukee, WI - [43.01599446681872, -87.92597233236194]
Parsed: 1329-A W ORCHARD ST, Milwaukee, WI - [43.01599253952926, -87.92892352553523]
Parsed: 1032 S 19TH ST, Milwaukee, WI - [43.02038767055225, -87.93674438631791]
Parsed: 1320 W BURNHAM ST, Milwaukee, WI - [43.01020514533914, -87.92847934187475]
Parsed: 553 W BOLIVAR AV, Milwaukee, WI - [42.966541688845474, -87.91863660476662]
Parsed: 126 W MARTIN

Parsed: 2545 N 29TH ST, Milwaukee, WI - [43.06491011518264, -87.94992808540556]
Parsed: 2400 W WRIGHT ST, Milwaukee, WI - [43.06427333289765, -87.94255576618741]
Parsed: 2639 N 18TH ST, Milwaukee, WI - [43.066826276992316, -87.93503059925536]
Parsed: 2802 N 23RD ST, Milwaukee, WI - [43.07000400000001, -87.94111542238471]
Parsed: 2508 W LLOYD ST, Milwaukee, WI - [43.05781048600596, -87.94546183236193]
Parsed: 1218 W CHERRY ST, Milwaukee, WI - [43.049956493219334, -87.92764178559067]
Parsed: 2026 N 39TH ST, Milwaukee, WI - [43.05709732944774, -87.96229644460172]
Parsed: 2376 N 44TH ST, Milwaukee, WI - [43.062308832361936, -87.96880238631792]
Parsed: 1200 W NORTH AV, Milwaukee, WI - [43.06042048600597, -87.92707997155058]
Parsed: 2807 N 29TH ST, Milwaukee, WI - [43.0699687543713, -87.94986013200395]
Parsed: 2025 W CENTER ST, Milwaukee, WI - [43.06780052120741, -87.93834367055226]
Parsed: 1611 N 28TH ST, Milwaukee, WI - [43.05171650291419, -87.94881957372014]
Parsed: 1036 W CENTER ST, Milw

Parsed: 4688 N 72ND ST, Milwaukee, WI - [43.10269241909515, -88.00190536078269]
Parsed: 6162 W MEDFORD AV, Milwaukee, WI - [43.09875584514389, -87.98927274949519]
Parsed: 5816 N 92ND ST, Milwaukee, WI - [43.12359874854292, -88.02701633524747]
Parsed: 5526 W FOND DU LAC AV, Milwaukee, WI - [43.094956870679106, -87.98257278945714]
Parsed: 7550 W PERKINS PL, Milwaukee, WI - [43.09090447936953, -88.00665148831457]
Parsed: 4069 N 71ST ST, Milwaukee, WI - [43.09151458673321, -88.00064165753918]
Parsed: 4440 N 76TH ST, Milwaukee, WI - [43.09820088481737, -88.00694492627986]
Parsed: 5700 N 87TH ST, Milwaukee, WI - [43.12143749125744, -88.02078294460172]
Parsed: 5141 N 84TH ST #4, Milwaukee, WI - [43.111432623925225, -88.01586894993916]
Parsed: 5506 N 76TH ST, Milwaukee, WI - [43.11784599163279, -88.00589387520941]
Parsed: 7034 W GRANTOSA DR, Milwaukee, WI - [43.11433334667309, -87.99929556676658]
Parsed: 5847 N 93RD ST, Milwaukee, WI - [43.124184000000014, -88.02826116475254]
Parsed: 6929 W SI

...

Parsed: 2330 W ROOSEVELT DR, Milwaukee, WI - [43.09570187901765, -87.94156672664302]
Parsed: 4167 N 20TH ST #UPR, Milwaukee, WI - [43.09318867055225, -87.93695865364403]
Parsed: 6908 N 37TH ST, Milwaukee, WI - [43.14302632127718, -87.95692698081955]
Parsed: 4947 N 22ND ST, Milwaukee, WI - [43.107354902971814, -87.9392680014589]
Parsed: 4545 N 29TH ST, Milwaukee, WI - [43.09998400582839, -87.94917762479058]
Parsed: 4170 N 22ND ST, Milwaukee, WI - [43.093502381903164, -87.93928396292358]
Parsed: 4183 N 21ST ST, Milwaukee, WI - [43.093593785734896, -87.93819154818492]
Parsed: 5155 N 28TH ST, Milwaukee, WI - [43.11094423535693, -87.9477677934096]
Parsed: 4771 N 21ST ST, Milwaukee, WI - [43.10400692200935, -87.93815913864039]
Parsed: 5000 N HOPKINS ST, Milwaukee, WI - [43.108332534784296, -87.96164933403662]
Parsed: 4765 N 32ND ST, Milwaukee, WI - [43.10312403136362, -87.95295760315051]
Parsed: 1907 W OLIVE ST, Milwaukee, WI - [43.09401598292576, -87.9344798785481]
Parsed: 4209 N 17TH ST, M

Parsed: 1327 W WASHINGTON ST, Milwaukee, WI - [43.02009054674261, -87.928821]
Parsed: 2175 S 16TH ST, Milwaukee, WI - [43.00491700582839, -87.93334108814686]
Parsed: 827 S 10TH ST, Milwaukee, WI - [43.02248399999999, -87.92393957314322]
Parsed: 907 S 1ST ST, Milwaukee, WI - [43.0215935342778, -87.91132849711447]
Parsed: 2444 S 14TH ST, Milwaukee, WI - [43.000271497085805, -87.9298164190665]
Parsed: 1818 S 10TH ST, Milwaukee, WI - [43.01051098251486, -87.92411668495032]
Parsed: 2115 S 13TH ST, Milwaukee, WI - [43.00605045045876, -87.92839048600597]
Parsed: 1820 S 16TH ST, Milwaukee, WI - [43.010612549541236, -87.93307247013695]
Parsed: 1400 W MINERAL ST, Milwaukee, WI - [43.02119450432783, -87.92945116763806]
Parsed: 800 S 3RD ST, Milwaukee, WI - [43.023181227924454, -87.91396009485494]
Parsed: 739 S 1ST ST, Milwaukee, WI - [43.02329332944774, -87.91127754760798]
Parsed: 1200 W GREENFIELD AV, Milwaukee, WI - [43.01709078463811, -87.92687801391814]
Parsed: 3311 S 18TH ST, Milwaukee, WI -

Parsed: 1040 E KNAPP ST, Milwaukee, WI - [43.04693952264971, -87.89836139064575]
Parsed: 2630 N HUMBOLDT BL, Milwaukee, WI - [43.066431639188664, -87.89781642238472]
Parsed: 1207 E KANE PL, Milwaukee, WI - [43.05605048124544, -87.896061]
Parsed: 2105 E KENWOOD BL, Milwaukee, WI - [43.07456252842077, -87.88381085789716]
Parsed: 2414 N BOOTH ST, Milwaukee, WI - [43.062461664723884, -87.90410290074465]
Parsed: 3319 N BREMEN ST, Milwaukee, WI - [43.078815115182636, -87.90020810646872]
Parsed: 2541 N OAKLAND AV #LWR, Milwaukee, WI - [43.06474858673323, -87.88806163200395]
Parsed: 2468 N PIERCE ST, Milwaukee, WI - [43.06355941326677, -87.90282490074463]
Parsed: 2920 N HUMBOLDT BL, Milwaukee, WI - [43.07178491035259, -87.89770645181508]
Parsed: 2962 N PIERCE ST, Milwaukee, WI - [43.07246941326679, -87.90274390074464]
Parsed: 1914 N PROSPECT AV, Milwaukee, WI - [43.054956315829074, -87.88873691975898]
Parsed: 2771 N PIERCE ST, Milwaukee, WI - [43.069004444630394, -87.90283857372015]
Parsed: 30

Parsed: 3018 N 1ST ST, Milwaukee, WI - [43.07346955536963, -87.91091590074464]
Parsed: 1100 E GARFIELD AV, Milwaukee, WI - [43.05920552928613, -87.89686166472387]
Parsed: 1719 W CAPITOL DR, Milwaukee, WI - [43.08943749567218, -87.93273086081135]
Parsed: 3745 N 6TH ST, Milwaukee, WI - [43.08496283819031, -87.91855010646873]
Parsed: 3041 N BOOTH ST, Milwaukee, WI - [43.0738657543713, -87.90400106650678]
Parsed: 3158 N ACHILLES ST, Milwaukee, WI - [43.075997465722196, -87.91004547345517]
Parsed: 3703 N 1ST ST, Milwaukee, WI - [43.084044922009355, -87.91097812089545]
Parsed: 3208 N BOOTH ST, Milwaukee, WI - [43.07687054954124, -87.90391390074464]
Parsed: 3334 N 9TH ST, Milwaukee, WI - [43.079696884817366, -87.92233441185313]
Parsed: 3626 N 16TH ST, Milwaukee, WI - [43.08408807799066, -87.93085291185312]
Parsed: 3839 N HUMBOLDT BL #204, Milwaukee, WI - [43.08719359282141, -87.89776365254751]
Parsed: 3839 N HUMBOLDT BL, Milwaukee, WI - [43.08719359282141, -87.89776365254751]
Parsed: 3835 N H

Parsed: 2600 W GREENFIELD AV, Milwaukee, WI - [43.01705364012725, -87.94644516170789]
Parsed: 737 S 36TH ST, Milwaukee, WI - [43.02275497446479, -87.9588980809335]
Parsed: 1308 S 35TH ST, Milwaukee, WI - [43.01746384401869, -87.95766304284747]
Parsed: 7415 W DEAN RD, Milwaukee, WI - [43.17052151659204, -88.00286290466843]
Parsed: 6060 W PORT AV, Milwaukee, WI - [43.1587294565756, -87.98585389508915]
Parsed: 8101 W KIEHNAU AV, Milwaukee, WI - [43.138868481245446, -88.01194997155058]
Parsed: 9324 W BROWN DEER RD, Milwaukee, WI - [43.17782148268776, -88.0270123097409]
Parsed: 10901 W DONNA DR, Milwaukee, WI - [43.17142254556013, -88.04635635036574]
Parsed: 5239 N 47TH ST, Milwaukee, WI - [43.112862335276134, -87.97135918307438]
Parsed: 5270 N 46TH ST, Milwaukee, WI - [43.11352741326678, -87.97008586078269]
Parsed: 7601 W DONNA CT, Milwaukee, WI - [43.17170054342441, -88.005829977783]
Parsed: 6102 W BIRCH AV, Milwaukee, WI - [43.11726645657561, -87.98740183236194]
Parsed: 5446 N 63RD ST, M